In [ ]:
# coding: utf-8

In [ ]:
"""
https://qiita.com/ysdyt/items/9ccca82fc5b504e7913a
⑥集計
"""

In [ ]:
"""
df.value_counts() #集計
df.groupby() #列ごとに集計.sum()や.mean()と組み合わせ
df.diff() #前行や前列との差分
df.rolling().mean() #移動平均~でもmean()が入ってるのでrolling自体は別のメソッドのような
df.pct_change() #前行や前列との変化率(差分＝A-B), 変化率＝(A-B)/B)
"""

In [ ]:
# ToDO
#  datetimeをindexに置き換える。
#  datetimeのdtypeを確認。必要であればdtypeをdatetimeに変更する。

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('lunch_box_4.csv')

In [6]:
df.set_index('datetime', inplace=True)
df.head()

,Unnamed: 0,sales,week,soldout,name,kcal,event,payday,weather,precipitation,temperature,month,days,period
datetime,,,,,,,,,,,,,,
2014-01-07,28,131,火,0,カレー入りソーセージカツ,404.0,NaN,0.0,快晴,0.0,7.3,1,7,上旬
2014-01-08,29,128,水,0,豚肉の生姜焼,462.0,NaN,0.0,曇,0.0,9.8,1,8,上旬
2014-01-09,30,129,木,1,鶏チリソース,435.0,NaN,0.0,曇,0.0,10.9,1,9,上旬
2014-01-10,31,87,金,0,手作りロースカツ,440.0,NaN,1.0,快晴,0.0,5.1,1,10,上旬
2014-01-14,32,129,火,1,鶏の照り焼きマスタード,376.0,NaN,0.0,晴れ,0.0,5.8,1,14,中旬


In [8]:
df.index

Index(['2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-14',
       '2014-01-15', '2014-01-16', '2014-01-17', '2014-01-20', '2014-01-21',
       ...
       '2014-09-11', '2014-09-16', '2014-09-17', '2014-09-18', '2014-09-19',
       '2014-09-22', '2014-09-24', '2014-09-25', '2014-09-29', '2014-09-30'],
      dtype='object', name='datetime', length=166)

In [9]:
df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
df.index

DatetimeIndex(['2014-01-07', '2014-01-08', '2014-01-09', '2014-01-10',
               '2014-01-14', '2014-01-15', '2014-01-16', '2014-01-17',
               '2014-01-20', '2014-01-21',
               ...
               '2014-09-11', '2014-09-16', '2014-09-17', '2014-09-18',
               '2014-09-19', '2014-09-22', '2014-09-24', '2014-09-25',
               '2014-09-29', '2014-09-30'],
              dtype='datetime64[ns]', name='datetime', length=166, freq=None)

In [10]:
#weather列の集計
df['weather'].value_counts()

曇     44
晴れ    41
快晴    34
薄曇    23
雨     22
雷電     1
雪      1
Name: weather, dtype: int64

In [11]:
#groupbyメソッドで、'week'列ごとに'soldoutの数をカウントする
df.groupby(['week'])['soldout'].count()

week
月    34
木    37
水    37
火    35
金    23
Name: soldout, dtype: int64

In [12]:
#groupメソッドは複数列に対しても行える
#groupメソッドで、'month', 'period'列ごとに'sales'の数を合計する
df.groupby(['month', 'period'])['sales'].sum()

month  period
1      上旬         475
       下旬        1018
       中旬         581
2      上旬         725
       下旬         609
       中旬         715
3      上旬         614
       下旬         369
       中旬         698
4      上旬         792
       下旬         457
       中旬         344
5      上旬         316
       下旬         517
       中旬         406
6      上旬         467
       下旬         311
       中旬         440
7      上旬         460
       下旬         406
       中旬         277
8      上旬         288
       下旬         334
       中旬         250
9      上旬         468
       下旬         234
       中旬         249
Name: sales, dtype: int64

In [14]:
#weatherごとにtemparatureの平均値を出す
df.groupby(['weather'])['temperature'].mean()

weather
快晴    15.294118
晴れ    22.558537
曇     19.377273
薄曇    23.926087
雨     18.813636
雪      1.200000
雷電    19.000000
Name: temperature, dtype: float64

In [15]:
#前行との差分が欲しい場合は.diff()を使う
df['temperature_diff'] = df['temperature'].diff(periods=1)
df[['temperature', 'temperature_diff']].head()
#比較する値との差: 引数periods　defaultは1。2だと例えば2行前と比較し差分を返す
#負の数も指定可能。-1のときは一行後ろとの差分（現在行-一行後ろ）
#引数axis=1で前列との差分になる。(おそらくdefaultはaxis=0)

,temperature,temperature_diff
datetime,,
2014-01-07,7.3,NaN
2014-01-08,9.8,2.5
2014-01-09,10.9,1.1
2014-01-10,5.1,-5.8
2014-01-14,5.8,0.7


In [17]:
# 'temperature'列について、頭からwindowsサイズ3で移動平均を計算する
df['temperature_rolling_mean'] = df['temperature'].rolling(window=3).mean()
df[['temperature', 'temperature_diff', 'temperature_rolling_mean']].head()

,temperature,temperature_diff,temperature_rolling_mean
datetime,,,
2014-01-07,7.3,NaN,NaN
2014-01-08,9.8,2.5,NaN
2014-01-09,10.9,1.1,9.333333
2014-01-10,5.1,-5.8,8.600000
2014-01-14,5.8,0.7,7.266667


In [23]:
#pct_change
#diff()がA - Bで差分を算出するのに対し、pct_change()は(A - B) / Bで変化率を算出する。
df['temperature_pct_change'] = df['temperature'].pct_change()
df[['temperature', 'temperature_diff', 'temperature_rolling_mean', 'temperature_pct_change']].head()

,temperature,temperature_diff,temperature_rolling_mean,temperature_pct_change
datetime,,,,
2014-01-07,7.3,NaN,NaN,NaN
2014-01-08,9.8,2.5,NaN,0.342466
2014-01-09,10.9,1.1,9.333333,0.112245
2014-01-10,5.1,-5.8,8.600000,-0.532110
2014-01-14,5.8,0.7,7.266667,0.137255


In [25]:
#ここまで計算したtemperature列たちに一つでもNaNがある行を削除
df.dropna(subset=['temperature', 'temperature_diff', 'temperature_rolling_mean', 'temperature_pct_change'], axis=0, inplace=True)

In [26]:
df.isnull().sum()

Unnamed: 0                    0
sales                         0
week                          0
soldout                       0
name                          0
kcal                          0
event                       153
payday                        0
weather                       0
precipitation                 0
temperature                   0
month                         0
days                          0
period                        0
temperature_diff              0
temperature_rolling_mean      0
temperature_pct_change        0
dtype: int64

In [27]:
df.head()

,Unnamed: 0,sales,week,soldout,name,kcal,event,payday,weather,precipitation,temperature,month,days,period,temperature_diff,temperature_rolling_mean,temperature_pct_change
datetime,,,,,,,,,,,,,,,,,
2014-01-09,30,129,木,1,鶏チリソース,435.0,NaN,0.0,曇,0.0,10.9,1,9,上旬,1.1,9.333333,0.112245
2014-01-10,31,87,金,0,手作りロースカツ,440.0,NaN,1.0,快晴,0.0,5.1,1,10,上旬,-5.8,8.600000,-0.532110
2014-01-14,32,129,火,1,鶏の照り焼きマスタード,376.0,NaN,0.0,晴れ,0.0,5.8,1,14,中旬,0.7,7.266667,0.137255
2014-01-15,33,134,水,0,さんま辛味焼,450.0,NaN,0.0,曇,0.0,2.9,1,15,中旬,-2.9,4.600000,-0.500000
2014-01-16,34,107,木,0,カレイ唐揚げ野菜あんかけ,415.0,NaN,0.0,快晴,0.0,7.0,1,16,中旬,4.1,5.233333,1.413793


In [29]:
#エクスポート
df.to_csv('lunch_box_5.csv')